# Geojson 전처리 과정 및 코드

## 전체 과정
1. QGIS (https://www.vw-lab.com/106 참고)
  - WMS 연결
  - 래스터화
  - GeoTiff형식으로 저장
  - 저장된 GeoTiff 데이터 불러오기
  - 이를 PCT로 변환
  - 단계별로 분류 후, 수치 값 지정하여 분류 (필드 명: DN)
  - 벡터 레이어로 변환
2. 데이터 전처리(Python)
  - Fid 관련 값 삭제
  - DN, multiplyDN, multiplyaDN_1 결합
  - QGIS 처리 OR 파이썬 처리 (택 1)
    - QGIS 처리 시, 색깔 지정 후 래스터 타일 mbtiles로 변환
    - 파이썬 처리 시 색깔 컬럼 추가 -> geoTiff -> mbtiles
  - +) 백터 타일 데이터 변환 코드
  

---
### Fid  관련 값 삭제

In [ ]:
import geopandas as gpd
import numpy as np
import os

In [ ]:
geojson_file_path = r"C:\Users\PC\OneDrive\바탕 화면\업무\dijiu\벡터통합지도.geojson" # 파일 경로
gpf = gpd.read_file(geojson_file_path)
columns_to_remove = ['fid', 'multiplyfid', 'multiplyfid_2']
gpf = gpf.drop(columns=columns_to_remove, errors='ignore')

# 정제된 데이터를 새로운 GeoJSON 파일로 저장
output_file_path = "nofid.geojson"
gpf.to_file(output_file_path, driver="GeoJSON")

### DN, multiplyDN, multiplyaDN_1 결합


In [ ]:
nofid_file_path = r"data/nofid.geojson" # 파일 경로
nofid_gpd = gpd.read_file(nofid_file_path)

In [ ]:
# 조건 1: 모든 컬럼이 null이 아닌 경우, 같은 행에서 최대값 추출
max_all_not_null = nofid_gpd.loc[
    nofid_gpd[["DN", "multiplyDN", "multiplyDN_2"]].notnull().all(axis=1),
    ["DN", "multiplyDN", "multiplyDN_2"]
].max(axis=1)

# 조건 2: multiplyDN_2 컬럼만 null인 경우, DN과 multiplyDN_1에서 최대값 추출
max_multiplyDN_2_null = nofid_gpd.loc[
    nofid_gpd["multiplyDN_2"].isnull() & nofid_gpd[["DN", "multiplyDN"]].notnull().all(axis=1),
    ["DN", "multiplyDN"]
].max(axis=1)

# 조건 3: DN 컬럼만 null인 경우, multiplyDN과 multiplyDN_2에서 최대값 추출
max_DN_null = nofid_gpd.loc[
    nofid_gpd["DN"].isnull() & nofid_gpd[["multiplyDN", "multiplyDN_2"]].notnull().all(axis=1),
    ["multiplyDN", "multiplyDN_2"]
].max(axis=1)

# 조건 4: multiplyDN 컬럼만 null인 경우, DN과 multiplyDN_2에서 최대값 추출
max_multiplyDN_1_null = nofid_gpd.loc[
    nofid_gpd["multiplyDN"].isnull() & nofid_gpd[["DN", "multiplyDN_2"]].notnull().all(axis=1),
    ["DN", "multiplyDN_2"]
].max(axis=1)

# 조건 5: DN 컬럼만 null이 아닌 경우 해당 값 추가
not_null_DN = nofid_gpd.loc[
    nofid_gpd["DN"].notnull() & nofid_gpd[["multiplyDN", "multiplyDN_2"]].isnull().all(axis=1),
    "DN"
]

# 조건 6: multiplyDN 컬럼만 null이 아닌 경우 해당 값 추가
not_null_multiplyDN = nofid_gpd.loc[
    nofid_gpd["multiplyDN"].notnull() & nofid_gpd[["DN", "multiplyDN_2"]].isnull().all(axis=1),
    "multiplyDN"
]
# 조건 7: multiplyDN 컬럼만 null이 아닌 경우 해당 값 추가
not_null_multiplyDN_2 = nofid_gpd.loc[
    nofid_gpd["multiplyDN_2"].notnull() & nofid_gpd[["DN", "multiplyDN"]].isnull().all(axis=1),
    "multiplyDN_2"
]

max_values = (
    max_all_not_null
    .append(max_DN_null)
    .append(max_multiplyDN_1_null)
    .append(max_multiplyDN_2_null)
    .append(not_null_DN)
    .append(not_null_multiplyDN)
    .append(not_null_multiplyDN_2)
).reset_index(drop=True)

In [ ]:
# 기존 데이터에서 geometry 정보 추출
geometries = nofid_gpd["geometry"].reset_index(drop=True)

In [ ]:
# 두 컬럼 결합
max_integrate = gpd.GeoDataFrame({"max_value": max_values, "geometry": geometries}, geometry="geometry")

# GeoJSON으로 저장
output_file_path = "data/max_integrate.geojson"
max_integrate.to_file(output_file_path, driver="GeoJSON")
print("GeoJSON 파일이 생성되었습니다: result.geojson")

### 색깔 컬럼 추가 -> geoTiff -> mbtiles

In [ ]:
# 필수 라이브러리 설치
!apt-get install -y gdal-bin python3-gdal python3-rasterio

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba

In [ ]:
# 파일 불러오기
geojson_file_path = r"/content/drive/MyDrive/map/data/max_integrate.geojson"
gdf = gpd.read_file(geojson_file_path)

# 좌표 4326으로 변환
gdf = gdf.to_crs(epsg=4326)

def get_color(value):
    if value == 30:
        return '#fdfbc7'
    elif value == 60:
        return '#e6ff99'
    elif value == 90:
        return '#38fefd'
    elif value == 120:
        return '#ce9afe'
    elif value == 150:
        return '#ce3f87'
    else:
        return '#eb8500'

from shapely.geometry import mapping

# 각 구역에 색상 부여
gdf['color'] = gdf['max_value'].apply(get_color)

# 업데이트 된 GeoJSON 파일 저장
updated_geojson_file = '/content/drive/MyDrive/map/data/max_color_integration.geojson'
gdf.to_file(updated_geojson_file, driver='GeoJSON')

In [ ]:
# 래스터화

# 'color' 컬럼의 값을 RGBA로 변환
gdf['color'] = gdf['color'].apply(lambda x: to_rgba(x))

# 변환할 범위 및 해상도 설정
transform = rasterio.transform.from_bounds(126.171238, 34.159190, 129.525233, 38.455383, 512, 512)

# 컬럼 값을 추출하고 각 채널을 분리
geoms = [(geom, (int(r * 255), int(g * 255), int(b * 255), int(a * 255))) for geom, (r, g, b, a) in zip(gdf.geometry, gdf['color'])]

# 빈 배열 생성
raster_r = np.zeros((512, 512), dtype=np.uint8)
raster_g = np.zeros((512, 512), dtype=np.uint8)
raster_b = np.zeros((512, 512), dtype=np.uint8)
raster_a = np.zeros((512, 512), dtype=np.uint8)

# 레스터화 및 채널별로 값 설정
for geom, (r, g, b, a) in geoms:
    rasterize([geom], out_shape=raster_r.shape, transform=transform, fill=0, out=raster_r, default_value=r, dtype='uint8', all_touched=True)
    rasterize([geom], out_shape=raster_g.shape, transform=transform, fill=0, out=raster_g, default_value=g, dtype='uint8', all_touched=True)
    rasterize([geom], out_shape=raster_b.shape, transform=transform, fill=0, out=raster_b, default_value=b, dtype='uint8', all_touched=True)
    rasterize([geom], out_shape=raster_a.shape, transform=transform, fill=0, out=raster_a, default_value=a, dtype='uint8', all_touched=True)

# 결과 시각화
raster = np.dstack((raster_r, raster_g, raster_b, raster_a))
plt.imshow(raster)
plt.show()

# 레스터 파일로 저장
with rasterio.open(
    '/content/drive/MyDrive/dimap/data/max_color.tif', 'w',
    driver='GTiff', height=raster_r.shape[0], width=raster_r.shape[1],
    count=4, dtype='uint8', crs='EPSG:4326',
    transform=transform,
) as dst:
    dst.write(raster_r, 1)
    dst.write(raster_g, 2)
    dst.write(raster_b, 3)
    dst.write(raster_a, 4)

In [ ]:
# GeoTIFF → MBTiles 변환
!gdal_translate -of MBTiles /content/drive/MyDrive/dimap/data/max_color.tif /content/drive/MyDrive/dimap/data/max_color.mbtiles

In [ ]:
# 줌레벨 8-15 타일 생성
!gdal2tiles.py -z 8-15 /content/drive/MyDrive/dimap/data/max_color.tif /content/drive/MyDrive/dimap/data/tiles

### +) 백터 타일 데이터 변환 코드

In [ ]:
# 설치
!apt-get install -y build-essential libsqlite3-dev zlib1g-dev
!git clone https://github.com/mapbox/tippecanoe.git
!cd tippecanoe && make -j
!tippecanoe/tippecanoe --version

In [ ]:
# 파일 불러오기
geojson_file_path = r"/content/drive/MyDrive/map/data/max_integrate.geojson"
gdf = gpd.read_file(geojson_file_path)

# 좌표 4326으로 변환
gdf = gdf.to_crs(epsg=4326)

def get_color(value):
    if value == 30:
        return '#fdfbc7'
    elif value == 60:
        return '#e6ff99'
    elif value == 90:
        return '#38fefd'
    elif value == 120:
        return '#ce9afe'
    elif value == 150:
        return '#ce3f87'
    else:
        return '#eb8500'

from shapely.geometry import mapping

# 각 구역에 색상 부여
gdf['color'] = gdf['max_value'].apply(get_color)

# 업데이트 된 GeoJSON 파일 저장
updated_geojson_file = '/content/drive/MyDrive/map/data/max_color_integration.geojson'
gdf.to_file(updated_geojson_file, driver='GeoJSON')

In [ ]:
# GeoJSON 파일을 MBTiles로 변환
mbtiles_file = '/content/drive/MyDrive/map/data/max_color_integration.mbtiles'

!tippecanoe/tippecanoe -o {mbtiles_file} --force --coalesce --minimum-zoom=8 --maximum-zoom=18 --bounds="125.068,34.0361,130.905,38.5479" {geojson_file}

print(f"MBTiles 파일이 다음 위치에 저장되었습니다: {mbtiles_file}")